In [1]:
import andi
import numpy as np
import csv as csv
import numpy as np

from sklearn.metrics import mean_absolute_error
import tensorflow as tf



import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras import losses, metrics

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import LSTM

from keras.layers import Dropout
from keras.layers import Bidirectional
from tensorflow.keras.models import load_model
import os
from matplotlib import pyplot as plt
from data_split import data_split

Using TensorFlow backend.


Importing the data

In [3]:
import csv
trajs_from_files = csv.reader(open('sub_more_traj/trajs_post2.txt','r'), delimiter=';', 
                                        lineterminator='\n',quoting=csv.QUOTE_NONNUMERIC)
validation = [[],[],[]]
for trajs in enumerate(trajs_from_files):
    validation[int(trajs[1][0])-1].append(trajs[1][1:])

Importing the networks, two (three) for inference of exponents and switching times, one for classification of first model, one for classification of second model

In [4]:

model_switch_a_t_new = load_model('nets/task3/1d/task3_new.h5')

model_switch_a_t_diff = load_model('nets/task3/1d/diff_task3.h5')

model_classi_first = load_model('nets/task3/1d/taks_3_classify_frst_checkout200.h5')

model_classi_sec = load_model('nets/task3/1d/taks_3_classify_second_checkout200.h5')

Personalized arctan to convert predictions into switching times stats

In [5]:
def my_atan(x1,x2):
    y=np.arctan2(x1,x2)
    b=y<0
    c=b.astype(int)*(2*np.pi)
    d=y+c 
    return    d;

# 2d data

Alternative version, it performed better. For both inference and model detection it is obtained by applying the 1d classifier and averging all dimension independently, we know this may not work well for LW

In [6]:
i=200
for dim in [2]:
    pred_hd_a_t=np.zeros((len(validation[1]),3))
    
    pred_hd_m1_first=np.zeros((len(validation[1]),5))
    
    pred_hd_m2_sec=np.zeros((len(validation[1]),5))
    
    valid_ch_dim=np.asarray(validation[dim-1])    #validation in the chosen dimension
    for jj in range(dim):
        x=valid_ch_dim[:,i*jj:i*(jj+1)]
        test_tim_step=np.arange(i)
        show_time_coll=np.tile(test_tim_step,(len(x),1))
        data_show,label_show,traj_show,times_show=data_split(x,
                                                             show_time_coll,
                                                                 labels=np.ones(len(x)),
                                                                 start_row=0,num_row=len(x),
                                                                 traj_len=x.shape[1],n_in=0,n_samples=1,
                                                                 p_p=1,hmin=0,hmax=2,limith=False,normalization=True)


        pred_a_t_new=model_switch_a_t_new(data_show)
        pr_t_new = my_atan(pred_a_t_new[:,2],pred_a_t_new[:,3])*200/(2*np.pi)
        pred_hd_a_t[:,:2]+=pred_a_t_new[:,:2]/dim
        pred_hd_a_t[:,2]+=pr_t_new/dim
        
        #model classification uses increments

        
        data_show_diff,label_show,traj_show_diff,times_show=data_split(np.diff(x,axis=1),
                                                             show_time_coll,
                                                                 labels=np.ones(len(x)),
                                                                 start_row=0,num_row=len(x),
                                                                 traj_len=x.shape[1]-1,n_in=0,n_samples=1,
                                                                 p_p=1,hmin=0,hmax=2,limith=False,normalization=True)

        #first model
        pred_m1_first=model_classi_first.predict(data_show_diff)
        pred_hd_m1_first+=pred_m1_first/dim
        
        
        #second model

        pred_m2_sec=model_classi_sec.predict(data_show_diff)
        pred_hd_m2_sec+=pred_m2_sec/dim
            

Assembling the total prediction array  in the format required by submission, giving (t,model 1,a1,model2,a2). For inference, only model new is used and its predictions are averaged in the different dimensions

In [7]:
predictions2d=np.zeros((len(validation[1]),5))
predictions2d[:,0]=pred_hd_a_t[:,2] # t switch
predictions2d[:,1]=np.argmax(pred_hd_m1_first,axis=1)
predictions2d[:,2]=pred_hd_a_t[:,0]  # a1
predictions2d[:,3]=np.argmax(pred_hd_m2_sec,axis=1)
predictions2d[:,4]=pred_hd_a_t[:,1]  #a2

In [8]:
np.save('predictions/task3/EXTRAtask3_predictions2d.npy',predictions2d)

In [10]:
pred_to_txt2d = 2*np.ones((len(predictions2d), 6))
for i in range(len(predictions2d)):
    for j in range(5):
        
        pred_to_txt2d[i, j+1] = predictions2d[i][j]
pred_to_txt = pred_to_txt2d
np.savetxt('predictions/task3/EXTRAtask3_predictions2d.txt', pred_to_txt.astype(float), fmt = '%1.5f', delimiter = ';')

# using the net trained on 2d for inference

In [12]:
model_a_t2d = tf.keras.models.load_model('nets/task3/2d/taks_3_2d_inference_200.h5')

In [13]:
def data_prepare_task3_inf(X,dim):
    import numpy as np 
    thr=1e-10
    X=np.asarray(X)
    print(X.shape)
    N=len(X)
    #compute length of each trajectory
    trj_len=int(X.shape[1]/dim)
    xvec=np.zeros((N,trj_len-1,dim+1))
    for d in range(dim):
        x = np.array(X[:,d*trj_len:(d+1)*trj_len])
        x = np.diff(x,axis=1) 
        sx = np.std(x,axis=1)
        xvec[:,:,d] = (x-np.mean(x,axis=1).reshape(len(x),1)) / np.where(sx>thr,sx,1).reshape(len(x),1)   # normalize x data
    xvec[:,:,dim]=np.arange(trj_len-1)/trj_len
    # regularize labels
    
    return xvec

In [14]:
x2d = data_prepare_task3_inf(validation[1],dim=2)

(2340, 400)


In [15]:
pred2d_u2d = model_a_t2d.predict(x2d)

In [16]:
pr_t_2d_u2d =  my_atan(pred2d_u2d[:,2],pred2d_u2d[:,3])*200/(2*np.pi)

In [17]:
predictions2d_alt=np.zeros((len(validation[1]),5))
predictions2d_alt[:,0]=pr_t_2d_u2d # t switch
predictions2d_alt[:,1]=np.argmax(pred_hd_m1_first,axis=1)
predictions2d_alt[:,2]=pred2d_u2d[:,0]  # a1
predictions2d_alt[:,3]=np.argmax(pred_hd_m2_sec,axis=1)
predictions2d_alt[:,4]=pred2d_u2d[:,1]  #a2

In [18]:
predictions2d_alt

array([[ 19.4372442 ,   4.        ,   0.74335992,   1.        ,
          0.50158453],
       [139.06960135,   3.        ,   1.63019156,   3.        ,
          1.27498281],
       [ 85.12016247,   4.        ,   1.58436382,   3.        ,
          1.50557339],
       ...,
       [194.78039979,   2.        ,   0.65542901,   2.        ,
          0.61219156],
       [193.62742781,   2.        ,   0.73403025,   4.        ,
          0.77530199],
       [  3.99988684,   2.        ,   0.59379697,   2.        ,
          0.66026318]])

In [19]:
np.save('predictions/task3/EXTRAtask3_predictions2dALT.npy',predictions2d_alt)